In [1]:
# Imports
from rgbmatrix import RGBMatrix
from flask_socketio import SocketIO
from ledflag.controller.controller import LedController
from ledflag.controller.modes.mode import Mode, Args
from ledflag.bridge.message import Instruction
from typing import Tuple


class TestArgs(Args):
    
    def __init__(self, c1: Tuple[int, int, int], c2: Tuple[int, int, int]):
        super().__init__()
        self.c1 = c1
        self.c2 = c2


class TestMode(Mode):
    
    def __init__(self, matrix: RGBMatrix, socketio: SocketIO):
        super().__init__(matrix, socketio)
    
    def run(self, args: TestArgs, **kwargs):
        free = kwargs['free']
        i = 0
        gradient = self.generate_gradient(args.c1, args.c2, i)
        while free():
            for y in range(i):
                for x in range(i):
                    self.matrix.SetPixel()
    
    @staticmethod
    def generate_gradient(c1: Tuple[int, int, int], c2: Tuple[int, int, int], length: int):
        step = tuple(map(lambda a, b: (b - a) / length, c1, c2))
        return [tuple(round(c1[i] + step[i] * n) for i in range(3)) for n in range(length)]
        

controller = LedController()
instruction = Instruction(TestMode, TestArgs())

controller.job_handler(instruction)